In [1]:
import math
import numpy as np
import pandas as pd
import time
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import RandomizedSearchCV

In [5]:
cct = pd.read_csv(r'C:\Users\uidn8348\Documents\My Received Files\Python\15. Projects\Project 1\Consumer_Complaints_train.csv')
ccts = pd.read_csv(r'C:\Users\uidn8348\Documents\My Received Files\Python\15. Projects\Project 1\Consumer_Complaints_test_share.csv')

In [6]:
ccts['Consumer disputed?'] = pd.Series([0]* len(ccts['Date received']))

In [7]:
cct['dtype'] = pd.Series(['train']*len(cct['Date received']))

In [8]:
ccts['dtype'] = pd.Series(['test']*len(ccts['Date received']))

In [9]:
ccts = pd.DataFrame(data = ccts)

In [10]:
ccts.tail()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Complaint ID,Consumer disputed?,dtype
119601,2012-09-15,Credit card,NaN,Closing/Cancelling account,NaN,NaN,NaN,JPMorgan Chase & Co.,IL,60538,NaN,NaN,Web,2012-09-21,Closed with monetary relief,Yes,153482,0,test
119602,2016-07-21,Credit reporting,NaN,Credit reporting company's investigation,No notice of investigation status/result,On XXXX/XXXX/16 a dispute and validation of a ...,Company has responded to the consumer and the ...,Experian,CA,956XX,NaN,Consent provided,Web,2016-07-21,Closed with explanation,Yes,2023523,0,test
119603,2015-06-09,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,"FAIR COLLECTIONS & OUTSOURCING, INC.",CA,90803,Servicemember,NaN,Phone,2015-06-12,Closed with explanation,Yes,1413678,0,test
119604,2014-02-26,Debt collection,Credit card,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,Encore Capital Group,PA,17764,NaN,NaN,Phone,2014-02-27,Closed with explanation,Yes,732458,0,test
119605,2015-02-11,Debt collection,"Other (i.e. phone, health club, etc.)",Cont'd attempts collect debt not owed,Debt is not mine,NaN,NaN,AllianceOne Recievables Management,CA,92103,NaN,NaN,Web,2015-02-11,Closed with explanation,Yes,1235226,0,test


In [11]:
cct = pd.DataFrame(data = cct)

In [12]:
All_data = cct.append(ccts)

In [13]:
cct.shape

(478421, 19)

In [14]:
All_data

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID,dtype
0,2014-05-15,Credit card,NaN,Billing statement,NaN,NaN,NaN,Wells Fargo & Company,MI,48342,Older American,NaN,Web,2014-05-16,Closed with explanation,Yes,No,856103,train
1,2014-09-18,Bank account or service,(CD) Certificate of deposit,"Making/receiving payments, sending money",NaN,NaN,NaN,Santander Bank US,PA,18042,NaN,NaN,Referral,2014-09-24,Closed,Yes,No,1034666,train
2,2014-03-13,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Equifax,CA,92427,NaN,NaN,Referral,2014-04-03,Closed with non-monetary relief,Yes,No,756363,train
3,2015-07-17,Credit card,NaN,Billing statement,NaN,"My credit card statement from US Bank, XXXX. X...",Company chooses not to provide a public response,U.S. Bancorp,GA,305XX,Older American,Consent provided,Web,2015-07-17,Closed with monetary relief,Yes,No,1474177,train
4,2014-11-20,Credit card,NaN,Transaction issue,NaN,NaN,NaN,Bank of America,MA,02127,NaN,NaN,Web,2014-11-28,Closed with explanation,Yes,No,1132572,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119601,2012-09-15,Credit card,NaN,Closing/Cancelling account,NaN,NaN,NaN,JPMorgan Chase & Co.,IL,60538,NaN,NaN,Web,2012-09-21,Closed with monetary relief,Yes,0,153482,test
119602,2016-07-21,Credit reporting,NaN,Credit reporting company's investigation,No notice of investigation status/result,On XXXX/XXXX/16 a dispute and validation of a ...,Company has responded to the consumer and the ...,Experian,CA,956XX,NaN,Consent provided,Web,2016-07-21,Closed with explanation,Yes,0,2023523,test
119603,2015-06-09,Debt collection,"Other (i.e. phone, health club, etc.)",Disclosure verification of debt,Not given enough info to verify debt,NaN,Company chooses not to provide a public response,"FAIR COLLECTIONS & OUTSOURCING, INC.",CA,90803,Servicemember,NaN,Phone,2015-06-12,Closed with explanation,Yes,0,1413678,test
119604,2014-02-26,Debt collection,Credit card,Disclosure verification of debt,Not given enough info to verify debt,NaN,NaN,Encore Capital Group,PA,17764,NaN,NaN,Phone,2014-02-27,Closed with explanation,Yes,0,732458,test


In [15]:
All_data.isnull().sum()

Date received                        0
Product                              0
Sub-product                     173225
Issue                                0
Sub-issue                       365685
Consumer complaint narrative    504376
Company public response         484859
Company                              0
State                             4764
ZIP code                          4774
Tags                            513950
Consumer consent provided?      428676
Submitted via                        1
Date sent to company                 0
Company response to consumer         0
Timely response?                     0
Consumer disputed?                   0
Complaint ID                         0
dtype                                0
dtype: int64

In [16]:
All_data.shape

(598027, 19)

In [17]:
for col in ['Sub-product', 'Sub-issue','Consumer complaint narrative','Company public response','Tags'
           ,'Consumer consent provided?']:
    varname = col.replace('-','_').replace('?','').replace(' ','_')+ '_Nan'
    All_data[varname] = np.where(pd.isnull(All_data[col]),1,0)
    All_data.drop([col],1,inplace = True)
       

In [18]:
np.where(All_data['Submitted via'].isnull() == True)

(array([561027], dtype=int64),)

In [19]:
value = 'Web'
All_data['Submitted via'] = All_data['Submitted via'].fillna(value)

In [20]:
All_data.shape

(598027, 19)

In [21]:
All_data['Date received'] = pd.to_datetime(All_data['Date received'],infer_datetime_format=True)

In [22]:
All_data['Date sent to company'] = pd.to_datetime(All_data['Date sent to company'],infer_datetime_format=True)


In [23]:
All_data['day_diff'] = pd.to_numeric(All_data['Date sent to company']-All_data['Date received'])


In [24]:
All_data.drop(['Date received'],1, inplace = True)
All_data.drop(['Date sent to company'],1,inplace = True)



In [25]:
All_data.drop(['Company'],1, inplace = True)
All_data.drop(['ZIP code'],1,inplace = True)


In [26]:
for col in ['Product','Issue','State','Submitted via','Company response to consumer','Timely response?']:
    
    freq = All_data[col].value_counts()
    select_catgory = freq.index[freq> 200][:-1]
    
    print(col)
    
    for cat in select_catgory:
        name = col+'_'+cat
        All_data[name] = (All_data[col]== cat).astype(int)
    del All_data[col] 

Product
Issue
State
Submitted via
Company response to consumer
Timely response?


In [27]:
cct = All_data.iloc[0:478421,]

In [28]:
cct.tail()

,Consumer disputed?,Complaint ID,dtype,Sub_product_Nan,Sub_issue_Nan,Consumer_complaint_narrative_Nan,Company_public_response_Nan,Tags_Nan,Consumer_consent_provided_Nan,day_diff,...,Submitted via_Referral,Submitted via_Phone,Submitted via_Postal mail,Submitted via_Fax,Company response to consumer_Closed with explanation,Company response to consumer_Closed with non-monetary relief,Company response to consumer_Closed with monetary relief,Company response to consumer_Closed without relief,Company response to consumer_Closed,Timely response?_Yes
478416,No,1445488,train,0,1,1,1,1,1,86400000000000,...,0,1,0,0,1,0,0,0,0,1
478417,No,438011,train,1,1,1,1,1,1,86400000000000,...,0,1,0,0,1,0,0,0,0,1
478418,No,1020651,train,1,0,1,1,1,1,0,...,0,0,0,0,1,0,0,0,0,1
478419,No,157524,train,0,1,1,1,1,1,259200000000000,...,0,0,0,0,1,0,0,0,0,1
478420,No,1093195,train,0,1,1,1,0,1,0,...,0,0,0,0,0,1,0,0,0,1


In [29]:
ccts = All_data.iloc[478421:,]

In [30]:
ccts.shape

(119606, 157)

In [31]:
cct['Consumer disputed?'] = np.where(cct['Consumer disputed?']== 'Yes',1,0)

C:\Users\uidn8348\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [32]:
x = cct.drop(['Complaint ID','Consumer disputed?','dtype'],1)

In [33]:
y = cct['Consumer disputed?']

In [34]:
ccts = ccts.drop(['Complaint ID','Consumer disputed?','dtype'],1)

In [43]:
from sklearn.metrics import roc_auc_score

In [44]:
from sklearn.ensemble import RandomForestClassifier

In [45]:
clf=RandomForestClassifier(n_estimators=100)

In [46]:
final_model = clf.fit(x,y)

In [47]:
ccts1 = All_data.iloc[478421:,]

In [48]:
ccts1=ccts1.drop(['Consumer disputed?','dtype'],1)

In [49]:
prediction=np.where(final_model.predict(ccts1.drop(['Complaint ID'],1))==1,"Yes","No")
final_prediction=pd.DataFrame(list(zip(ccts1['Complaint ID'],list(prediction))),
                columns=['Complaint ID','Consumer disputed?'])

In [54]:
final_prediction

,Complaint ID,Consumer disputed?
0,675956,No
1,1858795,No
2,32637,No
3,1731374,No
4,501487,No
...,...,...
119601,153482,No
119602,2023523,No
119603,1413678,No
119604,732458,No


In [57]:
final_prediction.to_csv('Anusha_P1.csv',index=False)

In [58]:
final_prediction

,Complaint ID,Consumer disputed?
0,675956,No
1,1858795,No
2,32637,No
3,1731374,No
4,501487,No
...,...,...
119601,153482,No
119602,2023523,No
119603,1413678,No
119604,732458,No
